In [1]:
import sys # funkcje systemowe, np. wyjscie z programu
import os # scieżki, funkcje systemowe itd.
import pyautogui # zewnetrzna biblioteka pip install pyautogui -> do zabawy z dźwiękiem
import tensorflow as tf # wgranie modelu sieci neuronowej
import numpy as np # operacje na macierzach
import cv2 # OpenCV2 -> computer vision


In [2]:
new_model = tf.keras.models.load_model("upvsdown2.model")

#parametry do cv2.putText()
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (255, 0, 0)
thickness = 2

CATEGORIES = ["up", "down"]

In [3]:
import math


class EuclideanDistTracker:
    
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids



In [4]:
#funkcja która pobiera zdjęcie i zwraca predykcje 
def frame_to_Network(imgArr):
    imgArrNew = imgArr.reshape(-1, 130, 130, 1) 
    prediction = new_model.predict(imgArrNew)
    label = np.argmax(prediction)
    
    return CATEGORIES[int(prediction[0][0])]




In [7]:

# inicjalizacja trackera
tracker = EuclideanDistTracker()
cap = cv2.VideoCapture(0)
object_detector = cv2.createBackgroundSubtractorMOG2(history=70, varThreshold=100)

while (True):
    
    #pobieramy klatka po klatce, i wrzucamy je do funkcji frame_to_Network
    ret, frame = cap.read()
    display = frame.copy()
    height, width, _ = frame.shape
    
    #roi
    roi = frame
    # robimy maske background subtracker, tego co sie porusza,
    # aby tracker mogl to wykryc
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    
    #resizujemy klatke na 130x130, oraz nakladamy filtr czarno-biały
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resizedImg = cv2.resize(gray, (130,130))
    
    
    #wypisujemy na ekran co wykryła nasza sieć "up lub down"    
    pred_text = cv2.putText(frame, frame_to_Network(resizedImg), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    
    
    #sprawdzenie czy nasza sieć wykryła up lub down, jeśli tak
    #to przyciszamy albo pogłaśniamy dźwięk
    

    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 1000:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)


            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

        
    cv2.imshow("Mask", mask)
    cv2.imshow('frame', frame)

    #wychodzimy z pętli za pomocą klawisza "Q" - quit
    k = cv2.waitKey(10)
    if k==ord('q'):
            break
    if k==ord('s'):
            if frame_to_Network(resizedImg) == "up":
                pyautogui.press("volumeup")
            if frame_to_Network(resizedImg) == "down":
                pyautogui.press("volumedown")

# wyłącz przechwytywanie
cap.release()
cv2.destroyAllWindows()


{2: (611, 147)}
{2: (616, 138)}
{3: (588, 251)}
{5: (615, 356), 6: (564, 359), 7: (453, 321), 8: (478, 389)}
{5: (615, 356), 6: (565, 358), 7: (453, 321), 8: (478, 389)}
{8: (479, 386), 5: (615, 356), 6: (565, 358), 9: (90, 348), 10: (383, 262)}
{8: (479, 386), 5: (615, 356), 6: (565, 358), 9: (90, 347), 10: (383, 262)}
{8: (478, 385), 9: (90, 347), 11: (302, 143)}
{8: (478, 385), 9: (90, 346), 11: (302, 143)}
{8: (478, 385), 9: (90, 346), 11: (311, 140)}
{8: (479, 385), 9: (90, 346), 11: (311, 140)}
{8: (479, 385), 9: (91, 343), 11: (311, 140), 12: (316, 347)}
{8: (479, 385), 9: (91, 343), 11: (312, 135), 12: (316, 347)}
{8: (480, 384), 12: (316, 347), 9: (91, 343), 11: (312, 135)}
{8: (480, 384), 12: (316, 347), 9: (91, 343), 11: (316, 133), 13: (90, 316), 14: (308, 234)}
{8: (480, 384), 13: (90, 315), 14: (308, 234), 11: (316, 133)}
{8: (480, 384), 13: (90, 315), 14: (309, 232), 11: (316, 133)}
{8: (480, 384), 13: (90, 315), 14: (309, 232), 11: (314, 129)}
{13: (90, 313), 14: (309, 

{121: (134, 386), 126: (529, 370), 127: (91, 352), 123: (340, 239), 128: (59, 248), 133: (205, 213), 134: (302, 99), 130: (432, 144)}
{121: (134, 386), 126: (530, 374), 127: (91, 352), 123: (340, 239), 128: (59, 248), 133: (205, 213), 134: (302, 99), 130: (432, 144)}
{121: (134, 386), 126: (530, 374), 127: (91, 351), 123: (340, 239), 128: (59, 248), 133: (205, 213), 134: (302, 99), 130: (432, 144)}
{121: (134, 386), 126: (530, 374), 127: (91, 351), 123: (340, 241), 128: (59, 248), 133: (205, 213), 134: (302, 99), 130: (432, 144), 135: (624, 245)}
{121: (134, 386), 126: (530, 374), 127: (91, 351), 123: (340, 241), 128: (59, 248), 133: (204, 229), 134: (302, 99), 130: (432, 144), 135: (624, 245)}
{121: (134, 386), 126: (530, 374), 127: (91, 351), 123: (340, 241), 128: (54, 252), 133: (204, 229), 134: (302, 99), 130: (432, 144), 135: (624, 245)}
{121: (134, 386), 126: (530, 374), 127: (91, 351), 123: (340, 241), 128: (54, 252), 133: (204, 229), 134: (293, 120), 130: (432, 144), 135: (624,

{200: (225, 349), 197: (59, 337)}
{200: (225, 349), 197: (58, 340)}
{200: (225, 349), 197: (59, 355), 201: (616, 382)}
{200: (241, 348), 197: (59, 355), 201: (616, 382)}
{201: (618, 382), 197: (59, 355), 200: (241, 348)}
{201: (618, 382), 197: (61, 354), 200: (241, 348)}
{201: (618, 382), 197: (61, 354), 200: (251, 348)}
{201: (616, 381), 197: (61, 354), 200: (251, 348)}
{201: (616, 381), 197: (61, 351), 200: (251, 348)}
{201: (616, 381), 197: (61, 351), 200: (251, 348)}
{201: (616, 381), 197: (55, 353), 200: (251, 348)}
{201: (616, 381), 197: (55, 353), 200: (248, 349), 202: (44, 323)}
{197: (55, 353), 202: (44, 323), 200: (261, 349), 203: (29, 377)}
{203: (16, 378), 200: (261, 349)}
{203: (16, 381), 204: (288, 378)}
{203: (20, 379), 205: (264, 349), 206: (326, 403)}
{206: (326, 403), 203: (16, 382)}
{208: (84, 356), 209: (169, 327)}
{208: (84, 356), 209: (157, 308)}
{208: (76, 364), 209: (157, 308)}
{208: (64, 376), 210: (206, 330), 211: (121, 242), 212: (36, 227)}
{208: (64, 376), 2